## [Exemples Utilisation GroqCloud](https://console.groq.com/docs/text-chat)

In [2]:
content = " 2 +2 ?"

In [3]:
import os
from dotenv import load_dotenv
from groq import Groq

# Load environment variables from .env file
load_dotenv()

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": content,
        }
    ],
    model="llama-3.1-70b-versatile",
)

print(chat_completion.choices[0].message.content)
# with open("information_api_key.md", "w") as f:
#     f.write(chat_completion.choices[0].message.content)

2 + 2 = 4.


In [16]:
from typing import List, Optional
import json

from pydantic import BaseModel
from groq import Groq

groq = Groq()


# Data model for LLM to generate
class Ingredient(BaseModel):
    name: str
    quantity: str
    quantity_unit: Optional[str]


class Recipe(BaseModel):
    recipe_name: str
    ingredients: List[Ingredient]
    directions: List[str]


def get_recipe(recipe_name: str) -> Recipe:
    chat_completion = groq.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "You are a recipe database that outputs recipes in JSON.\n"
                # Pass the json schema to the model. Pretty printing improves results.
                f" The JSON object must use the schema: {json.dumps(Recipe.model_json_schema(), indent=2)}",
            },
            {
                "role": "user",
                "content": f"Fetch a recipe for {recipe_name}",
            },
        ],
        model="llama3-8b-8192",
        temperature=0,
        # Streaming is not supported in JSON mode
        stream=False,
        # Enable JSON mode by setting the response format
        response_format={"type": "json_object"},
    )
    return Recipe.model_validate_json(chat_completion.choices[0].message.content)


def print_recipe(recipe: Recipe):
    print("Recipe:", recipe.recipe_name)

    print("\nIngredients:")
    for ingredient in recipe.ingredients:
        print(
            f"- {ingredient.name}: {ingredient.quantity} {ingredient.quantity_unit or ''}"
        )
    print("\nDirections:")
    for step, direction in enumerate(recipe.directions, start=1):
        print(f"{step}. {direction}")


recipe = get_recipe("croissant aux amandes in French")
print_recipe(recipe)


Recipe: Croissant aux Amandes

Ingredients:
- Farine: 500g grammes
- Pâte à beurre: 250g grammes
- Sucre: 20g grammes
- Sel: 5g grammes
- Œufs: 2 unités
- Amandes: 100g grammes
- Beurre fondu: 100g grammes

Directions:
1. Mélangez la farine, le sucre et le sel dans un bol.
2. Ajoutez la pâte à beurre et mélangez jusqu'à obtenir une pâte homogène.
3. Ajoutez les œufs un à un et mélangez jusqu'à obtenir une pâte lisse.
4. Ajoutez les amandes et le beurre fondu et mélangez jusqu'à obtenir une pâte homogène.
5. Faites reposer la pâte pendant 1 heure.
6. Déroulez la pâte et faites-la reposer pendant 30 minutes.
7. Faites cuire les croissants pendant 20-25 minutes à 180°C.
